In [1]:
import numpy as np
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import torch.optim.lr_scheduler as lr_scheduler

## Assigning to GPU

In [2]:
# Check if GPU is available and set the device accordingly
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#print(f"Using device: {device}")

## Load Dataset

In [3]:
# Load the dataset
df = pd.read_csv(r"C:\Users\HP\Desktop\DeepLearning_Assignment1_Housing\Housing-1 - Housing-1.csv")

In [ ]:
# Display first few rows
print("First few rows of the dataset:")
df.head()

First few rows of the dataset:


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [5]:
df.nunique()

price               219
area                284
bedrooms              6
bathrooms             4
stories               4
mainroad              2
guestroom             2
basement              2
hotwaterheating       2
airconditioning       2
parking               4
prefarea              2
furnishingstatus      3
dtype: int64

## Check total count of items and notna and compare
If they are the same then no need to drop na 

In [6]:
df.count()

price               545
area                545
bedrooms            545
bathrooms           545
stories             545
mainroad            545
guestroom           545
basement            545
hotwaterheating     545
airconditioning     545
parking             545
prefarea            545
furnishingstatus    545
dtype: int64

In [7]:
df.notna().sum()

price               545
area                545
bedrooms            545
bathrooms           545
stories             545
mainroad            545
guestroom           545
basement            545
hotwaterheating     545
airconditioning     545
parking             545
prefarea            545
furnishingstatus    545
dtype: int64

In [8]:
# Step 2: Encoding Categorical Variables
# Use OneHotEncoder for categorical features
# Step 2: Encoding Categorical Variables
# Use OneHotEncoder for categorical features
categorical_cols = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 
                    'airconditioning', 'prefarea', 'furnishingstatus']

# Corrected OneHotEncoder usage
encoder = OneHotEncoder(sparse_output=False, drop='first')  # drop='first' to avoid dummy variable trap
encoded_categorical = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(encoded_categorical, columns=encoder.get_feature_names_out(categorical_cols))


# Step 3: Combine encoded categorical columns with the original DataFrame
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Step 4: Drop the original categorical columns from the DataFrame
df.drop(categorical_cols, axis=1, inplace=True)

# Step 5: Feature Scaling
scaler = StandardScaler()
numerical_cols = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking']
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [9]:
df

,price,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,1.046726,1.403419,1.421812,1.378217,1.517692,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
1,12250000,1.757010,1.403419,5.405809,2.532024,2.679409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,12250000,2.218232,0.047278,1.421812,0.224410,1.517692,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,12215000,1.083624,1.403419,1.421812,0.224410,2.679409,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
4,11410000,1.046726,1.403419,-0.570187,0.224410,1.517692,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,-0.991879,-1.308863,-0.570187,-0.929397,1.517692,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
541,1767150,-1.268613,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
542,1750000,-0.705921,-1.308863,-0.570187,-0.929397,-0.805741,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
543,1750000,-1.033389,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Data Cleaning Process for Outlier Detection

In this code snippet, we perform several steps to clean the dataset by removing outliers based on the target variable, which is the price of a property. Here’s a breakdown of what happens:

1. **Feature and Target Variable Extraction**:
   - We extract the feature (`area`) and the target variable (`price`) from the DataFrame `df`. The feature `X` is reshaped into a 2D array with one column to ensure compatibility with certain machine learning models, while the target variable `y` is retained as a 1D array.

2. **Outlier Detection Using IQR**:
   - We calculate the first (Q1) and third (Q3) quartiles of the target variable `y`. 
   - The Interquartile Range (IQR) is computed as the difference between Q3 and Q1. This IQR helps identify the spread of the middle 50% of the data.
   - Using the IQR, we set lower and upper bounds for outlier detection: any data point falling below the lower bound (`Q1 - 1.5 * IQR`) or above the upper bound (`Q3 + 1.5 * IQR`) is considered an outlier.






In [10]:
# Calculate the first quartile (Q1) of the target variable 'price'
Q1 = np.percentile(np.array(df['price']).reshape(-1,1), 25)

# Calculate the third quartile (Q3) of the target variable 'price'
Q3 = np.percentile(np.array(df['price']).reshape(-1,1), 75)

# Calculate the Interquartile Range (IQR)
IQR = Q3 - Q1  # The IQR represents the range between Q1 and Q3

# Determine the lower and upper bounds for detecting outliers
lower_bound = Q1 - 1.5 * IQR  # Lower bound for outlier detection
upper_bound = Q3 + 1.5 * IQR  # Upper bound for outlier detection

# Clean the dataset by filtering out rows where the 'price' is an outlier
# Only include rows where 'price' is between the lower and upper bounds
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

# Update X and y with the cleaned data
X = df.drop('price', axis=1) # Reassign X with the cleaned 'area' data
y = np.array(df['price'])  # Reassign y with the cleaned 'price' data

X

,area,bedrooms,bathrooms,stories,parking,mainroad_yes,guestroom_yes,basement_yes,hotwaterheating_yes,airconditioning_yes,prefarea_yes,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
15,0.391790,1.403419,-0.570187,0.224410,1.517692,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
16,0.668524,1.403419,1.421812,0.224410,0.355976,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
17,1.544847,0.047278,1.421812,2.532024,1.517692,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
18,-0.253922,0.047278,1.421812,0.224410,1.517692,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
19,0.585503,0.047278,1.421812,0.224410,0.355976,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,-0.991879,-1.308863,-0.570187,-0.929397,1.517692,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
541,-1.268613,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
542,-0.705921,-1.308863,-0.570187,-0.929397,-0.805741,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
543,-1.033389,0.047278,-0.570187,-0.929397,-0.805741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
X.dtypes

area                               float64
bedrooms                           float64
bathrooms                          float64
stories                            float64
parking                            float64
mainroad_yes                       float64
guestroom_yes                      float64
basement_yes                       float64
hotwaterheating_yes                float64
airconditioning_yes                float64
prefarea_yes                       float64
furnishingstatus_semi-furnished    float64
furnishingstatus_unfurnished       float64
dtype: object

In [12]:
# Train-test split: 60% train , 40% val + test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)
# Train-test split: 20% val, 20% test
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)
X_val=scaler.transform(X_val)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
y_train = scaler.fit_transform(y_train)
y_val = scaler.transform(y_val)
y_test = scaler.transform(y_test)

In [13]:
# Convert data to PyTorch tensors
X_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).float()
X_val = torch.from_numpy(X_val).float()
y_val = torch.from_numpy(y_val).float()
X_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).float()

# Implementing a neural network for regression

In [14]:
# Neural network model
class HousingModel(nn.Module):
    def __init__(self, input_size):
        super(HousingModel, self).__init__()
        self.linear = nn.Linear(input_size,128)
        self.dropout1 = nn.Dropout(0.2)
        self.relu = nn.ReLU()
        self.linear2 = nn.Linear(128,64)
        self.dropout2 = nn.Dropout(0.2)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(64,16)
        self.dropout3 = nn.Dropout(0.2)
        self.relu3 = nn.ReLU()
        self.output = nn.Linear(16,1)

    def forward(self, x):
        x = self.linear(x)
        x = self.dropout1(x)
        x = self.relu(x)
        x = self.linear2(x)
        x = self.dropout2(x)
        x = self.relu2(x)
        x = self.linear3(x)
        x = self.dropout3(x)
        x = self.relu3(x)
        return self.output(x)

# Training and Validation for Model Evaluation


In [15]:
# Define parameters and instantiate model
epochs=10000
model = HousingModel(input_size=13)
criterion = nn.MSELoss()  # Use MSE for regression
optimizer = optim.SGD(model.parameters(), lr=0.07)
# Define the learning rate scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.2)

In [16]:
# Training loop
losses = []
val_losses = []  # To track validation losses
for epoch in range(epochs):
    # --- Training Phase ---
    model.train()  # Set model to training mode
    
    # Forward pass
    y_pred = model(X_train)  # Training prediction

    # Compute the loss
    loss = criterion(y_pred, y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # Track training loss
    losses.append(loss.item())

    # --- Validation Phase ---
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # Disable gradient calculation for validation
        y_val_pred = model(X_val)  # Validation prediction
        val_loss = criterion(y_val_pred, y_val)  # Validation loss

        # Track validation loss
        val_losses.append(val_loss.item())
    scheduler.step()
    # Print the results every 100 epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], '
              f'Train Loss: {loss.item():.4f}, '
              f'Val Loss: {val_loss.item():.4f}')

Epoch [100/10000], Train Loss: 0.3640, Val Loss: 0.5297
Epoch [200/10000], Train Loss: 0.2598, Val Loss: 0.5267
Epoch [300/10000], Train Loss: 0.2744, Val Loss: 0.5320
Epoch [400/10000], Train Loss: 0.2258, Val Loss: 0.5294
Epoch [500/10000], Train Loss: 0.2217, Val Loss: 0.5300
Epoch [600/10000], Train Loss: 0.1625, Val Loss: 0.5404
Epoch [700/10000], Train Loss: 0.1567, Val Loss: 0.5390
Epoch [800/10000], Train Loss: 0.1503, Val Loss: 0.5436
Epoch [900/10000], Train Loss: 0.1547, Val Loss: 0.5529
Epoch [1000/10000], Train Loss: 0.1422, Val Loss: 0.5487
Epoch [1100/10000], Train Loss: 0.1286, Val Loss: 0.5448
Epoch [1200/10000], Train Loss: 0.1286, Val Loss: 0.5435
Epoch [1300/10000], Train Loss: 0.1462, Val Loss: 0.5420
Epoch [1400/10000], Train Loss: 0.1260, Val Loss: 0.5404
Epoch [1500/10000], Train Loss: 0.1170, Val Loss: 0.5418
Epoch [1600/10000], Train Loss: 0.1179, Val Loss: 0.5381
Epoch [1700/10000], Train Loss: 0.1285, Val Loss: 0.5438
Epoch [1800/10000], Train Loss: 0.1344, 

## Model Evaluation with No Gradient Calculation

The following code evaluates the performance of a trained model on the test dataset by computing common regression metrics like Mean Squared Error (MSE), Mean Absolute Error (MAE), and the R-squared (R²) score.


In [17]:
# Evaluation with no gradient calculation
with torch.no_grad():
    y_pred_test = model(X_test)
    # Compute the metrics
    mse = mean_squared_error(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    r2 = r2_score(y_test, y_pred_test)

    # Print the results
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"R-squared (R2): {r2:.4f}")


Mean Squared Error (MSE): 0.4285
Mean Absolute Error (MAE): 0.4679
R-squared (R2): 0.6745


# Model Testing Function

In [18]:
# Evaluate the model on the test set
with torch.no_grad():
    y_pred_test = model(X_test)
    test_loss = criterion(y_pred_test, y_test)
    print(f'Test Loss: {test_loss.item():.4f}')

Test Loss: 0.4285
